In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [2]:
os.environ["NCCL_P2P_DISABLE"] = "1"


In [3]:
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'

In [4]:
os.environ["VLLM_USE_V1"] = "1"


In [5]:
import vllm
from vllm import LLM, SamplingParams
import pickle
import time
import os
import numpy as np
import json

INFO 05-24 15:44:47 [__init__.py:239] Automatically detected platform cuda.


In [6]:
model_id = "/data/just/models/qwen3_30b_a3b"

In [7]:
default_params = {
    #     "trust_remote_code": True,
    #     "tensor_parallel_size": len(os.environ["CUDA_VISIBLE_DEVICES"].split(",")), libcuda.so.1
        "gpu_memory_utilization": 0.9,
        # "dtype": 'bfloat16',
        'max_model_len': 32000,
        'tensor_parallel_size' : len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))
    }

In [8]:
llm = LLM(
        model=model_id,
        **default_params
    )

INFO 05-24 15:44:52 [config.py:2968] Downcasting torch.float32 to torch.float16.
INFO 05-24 15:44:58 [config.py:717] This model supports multiple tasks: {'classify', 'score', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 05-24 15:44:58 [config.py:1770] Defaulting to use mp for distributed inference
INFO 05-24 15:44:58 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-24 15:45:01 [__init__.py:239] Automatically detected platform cuda.
INFO 05-24 15:45:04 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='/data/just/models/qwen3_30b_a3b', speculative_config=None, tokenizer='/data/just/models/qwen3_30b_a3b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32000, download_dir=None, load_format=auto, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quanti

Loading safetensors checkpoint shards:   0% Completed | 0/25 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   4% Completed | 1/25 [00:00<00:07,  3.36it/s]
Loading safetensors checkpoint shards:   8% Completed | 2/25 [00:00<00:07,  3.08it/s]
Loading safetensors checkpoint shards:  12% Completed | 3/25 [00:01<00:07,  2.82it/s]
Loading safetensors checkpoint shards:  16% Completed | 4/25 [00:01<00:07,  2.82it/s]
Loading safetensors checkpoint shards:  20% Completed | 5/25 [00:01<00:06,  2.94it/s]
Loading safetensors checkpoint shards:  24% Completed | 6/25 [00:02<00:06,  3.01it/s]
Loading safetensors checkpoint shards:  28% Completed | 7/25 [00:02<00:06,  2.88it/s]
Loading safetensors checkpoint shards:  32% Completed | 8/25 [00:02<00:06,  2.76it/s]
Loading safetensors checkpoint shards:  36% Completed | 9/25 [00:03<00:05,  2.73it/s]
Loading safetensors checkpoint shards:  40% Completed | 10/25 [00:03<00:05,  2.66it/s]
Loading safetensors checkpoint shards:  44% Completed | 11/25

(VllmWorker rank=5 pid=1189800) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.27 seconds
(VllmWorker rank=4 pid=1189799) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.29 seconds
(VllmWorker rank=0 pid=1189795) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.28 seconds
(VllmWorker rank=2 pid=1189797) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.29 seconds
(VllmWorker rank=7 pid=1189802) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.30 seconds
(VllmWorker rank=3 pid=1189798) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.27 seconds
(VllmWorker rank=6 pid=1189801) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.30 seconds
(VllmWorker rank=1 pid=1189796) INFO 05-24 15:45:41 [loader.py:458] Loading weights took 28.36 seconds
(VllmWorker rank=5 pid=1189800) INFO 05-24 15:45:41 [gpu_model_runner.py:1347] Model loading took 7.1631 GiB and 28.702761 seconds
(VllmWorker rank=4 pid=1189799) INFO 05-24 15

In [9]:
sampling_params = SamplingParams(temperature=0.6, 
                         top_p=0.95, top_k=40,
                             max_tokens=82500)

In [10]:
def apply_qwen_math_template(question: str):
    return (
        "<|im_start|>system\nYou are a helpful and harmless assistant. You should think step-by-step. Think and respond entirely in English. You do not face any time or effort limitations.<|im_end|>\n<|im_start|>user\n"
        + question
        + "<|im_end|>\n<|im_start|>assistant\n"
    )

In [11]:
def apply_qwen_math_template(question: str):
    return (
        f"<｜begin▁of▁sentence｜><｜User｜>{question}<｜Assistant｜><think>\n"

    )

In [12]:
def get_multiple_model_answer(prompts):
        new_prompts = []
    
        # for prompt in prompts:
        #     messages = [
        #         {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step. Think and respond entirely in English. You do not face any time or effort limitations."},
        #         # {"role": "system", "content": "You are a knowledgeable, efficient, and direct AI assistant."},
        #         {"role": "user", "content": prompt},
        #     ]
        #     new_prompts.append(messages)

        new_prompts = list(map(apply_qwen_math_template, prompts))
        # out = llm.chat(new_prompts, sampling_params)
        out = llm.generate(new_prompts, sampling_params)
        return out

In [13]:
datasets = ["ARC_question",
            "BBH_logical3_question",
            "TAL_question",
            "BBH_logical5_tag_question",
           "BBH_logical5_basic_question",
           "BBH_logical3_tag_question",
           "BBH_date_tag_question",
           "BBH_date_basic_question",
           "ARC_tag_question",
           "TAL_tag_question",
           "MMLU_college_medicine_tag_question",
           "MMLU_high_physics_tag_question",
           "MMLU_college_physics_tag_question",
           "MMLU_college_medicine_basic_question",
           "MMLU_high_physics_basic_question",
           "MMLU_college_physics_basic_question"]

In [14]:
for datasetname in datasets:

    print(datasetname)
    with open(datasetname + ".json" , "r") as f:
        dataset = json.load(f)
    print(len(dataset))
    
    dataset_questions = []
    max_len = min(len(dataset), 300)
    print(f"{len(dataset)} dataset max len : {max_len}")
    for i in range(max_len):
        dataset_questions.append(dataset[i]['ambiguous question'])

    
    test_answers_gen_t06 = []
    start_t = time.time()
    block_size = 20
    
    for j in range(0, len(dataset_questions), block_size):
    # for j in range(0, 1, 1):
        cur_questions = dataset_questions[j:j+block_size]
        outs = get_multiple_model_answer(cur_questions)
        
        for k in range(len(cur_questions)):
            cur_outs = outs[k].outputs[0].text
            test_answers_gen_t06.append(cur_outs)
    
    
        # if j % 50 == 0:
        cur_t = time.time()
        print(f"{j+block_size}/{len(dataset_questions)} took {cur_t-start_t} seconds." )
    with open(f"qwen3_30b_a3b_{datasetname}_ambiguous_responses_{max_len}.json", "w") as f:
        json.dump(test_answers_gen_t06, f)

ARC_question
1119
1119 dataset max len : 300


Processed prompts:   0%|          | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

20/300 took 118.8571765422821 seconds.


Processed prompts:   0%|          | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

40/300 took 218.83460783958435 seconds.


...

120/151 took 1404.3813495635986 seconds.


Processed prompts:   0%|          | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

140/151 took 1618.5586721897125 seconds.


Processed prompts:   0%|          | 0/11 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

160/151 took 1789.3452005386353 seconds.
